In [1]:
# Install Dependencies
!pip install polyglot
!pip install pyicu
!pip install pycld2
!pip install morfessor
!pip install wordcloud
!pip install seaborn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.3/126.3 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for polyglot: filename=polyglot-16.7.4-py2.py3-none-any.whl size=52561 sha256=1d174e2e34bb4a3940c7536c3b9a2b8ac299f5872754dd2bdb61a6906d2e7770
  Stored in directory: /root/.cache/pip/wheels/aa/92/4a/b172589446ba537db3bdb9a1f2204f27fe71217981c14ac368
Successfully built polyglot
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.4/262.4 kB 10.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyicu: filename=PyICU-2.13.1-cp310-cp310-linux_x86_64.whl size=1801403 sha256=1a9475ec887689a0e8815dd5bb5227f8b869d13fac2f2aadec22ed3ae3599b94
  Stored in directory: /root/.cache/pip/wheels/8e/45/08/f4bc505e31eb7bb779d493226921abf18cf55ed30142e70eae
Successfully built pyicu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 MB 27.4 MB/s e

In [2]:
!polyglot download pos2.id
!polyglot download embeddings2.id

[polyglot_data] Downloading package pos2.id to /root/polyglot_data...
[polyglot_data] Downloading package embeddings2.id to
[polyglot_data]     /root/polyglot_data...


In [3]:
import pandas as pd
from polyglot.text import Text
from collections import Counter
import re

In [4]:
from google.colab import files
uploaded = files.upload()

Saving strava_reviews_df_after_stopwords_stemming.csv to strava_reviews_df_after_stopwords_stemming.csv


In [6]:
file_name = list(uploaded.keys())[0]
strava_reviews_df = pd.read_csv(file_name)
strava_reviews_df.head()

,content,score,sentiment_polarity,sentiment_subjective,sentiment_rating,review_without_stopwords,wordCount,wordCount_after_stopwords
0,aplikasi yg sangat bagus,5,0.0,0.0,Positive,aplikasi yg bagus,4,3
1,belum support bahasa indonesia,3,0.0,0.0,Neutral,support bahasa indonesia,4,3
2,good,5,0.7,0.6,Positive,good,1,1
3,sangat membantu min,4,0.0,0.0,Positive,membantu min,3,2
4,"strava strava,map jaman kapan yg di pakai ini?...",1,0.0,0.0,Negative,"strava strava,map jaman yg pakai ini? map jama...",14,12


In [8]:
import pandas as pd
import re

def clean_text(text):
    # Handle float values (e.g., NaNs) by converting them to empty strings
    if isinstance(text, float):
        return ''
    # Convert to lowercase, remove special characters, and extra spaces
    text = re.sub(r'[^\w\s]', '', text.lower())
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Apply cleaning to the 'content' column of Alo

In [9]:
strava_reviews_df

,content,score,sentiment_polarity,sentiment_subjective,sentiment_rating,review_without_stopwords,wordCount,wordCount_after_stopwords
0,aplikasi yg sangat bagus,5,0.000,0.000000,Positive,aplikasi yg bagus,4,3
1,belum support bahasa indonesia,3,0.000,0.000000,Neutral,support bahasa indonesia,4,3
2,good,5,0.700,0.600000,Positive,good,1,1
3,sangat membantu min,4,0.000,0.000000,Positive,membantu min,3,2
4,"strava strava,map jaman kapan yg di pakai ini?...",1,0.000,0.000000,Negative,"strava strava,map jaman yg pakai ini? map jama...",14,12
...,...,...,...,...,...,...,...,...
4797,nice apps,5,0.600,1.000000,Positive,nice apps,2,2
4798,pk,5,0.000,0.000000,Positive,pk,1,1
4799,this free version app is more than enough for ...,4,0.325,0.608333,Positive,this free version app is more than enough for ...,19,19
4800,asyik,5,0.000,0.000000,Positive,asyik,1,1


In [12]:
import pandas as pd
from collections import Counter

# Store all POS tags and words
all_words = []
all_pos_tags = []
word_pos_map = {}

# Loop through each review in 'wordCount_after_stopwords' column
for content in strava_reviews_df['wordCount_after_stopwords']:
    # Convert content to string before applying strip()
    content_str = str(content)
    if content_str.strip():  # Check if content is not empty
        text = Text(content_str, hint_language_code='id')  # 'id' indicates Indonesian
        pos_tags = text.pos_tags

        # Collect words and their corresponding POS tags
        for word, pos in pos_tags:
            word_lower = word.lower()
            all_words.append(word_lower)
            all_pos_tags.append(pos)

            # Map each POS tag with the corresponding word
            if pos in word_pos_map:
                word_pos_map[pos].append(word_lower)
            else:
                word_pos_map[pos] = [word_lower]

# Calculate count of POS tags
pos_counts = Counter(all_pos_tags)

# Calculate unique tokens for each POS tag
unique_tokens_count = {pos: len(set(words)) for pos, words in word_pos_map.items()}

# Prepare data for the DataFrame
tag_data = []
for pos, count in pos_counts.items():
    unique_tokens = unique_tokens_count.get(pos, 0)
    tag_data.append({'Tag': pos, 'Count': count, 'Unique Tokens': unique_tokens})

# Create DataFrame
pos_df = pd.DataFrame(tag_data)

# Display the DataFrame
print(pos_df)

     Tag  Count  Unique Tokens
0    NUM   4233             10
1  PROPN    569             43


In [13]:
pos_df.to_csv('pos_tag_strava.csv', index=False)

In [17]:
# Count word frequency
word_counts = Counter(all_words)

# Count POS tag frequency
pos_counts = Counter(all_pos_tags)

# Display the most common words and POS tags
print("Top 10 most common words:", word_counts.most_common(10))
print("Top 10 most common POS tags:", pos_counts.most_common(10))

Top 10 most common words: [('1', 1770), ('2', 796), ('3', 460), ('4', 334), ('5', 273), ('6', 206), ('7', 144), ('8', 108), ('9', 99), ('10', 91)]
Top 10 most common POS tags: [('NUM', 4233), ('PROPN', 569)]


In [19]:
word_info = []

for word, freq in word_counts.items():
    # Find the POS tag for the word
    pos_tag = None
    for content in strava_reviews_df['wordCount_after_stopwords']:
        # Check if content is a string before applying strip()
        if isinstance(content, str):  # Check if content is a string
            if content.strip():
                text = Text(content, hint_language_code='id')
                for w, pos in text.pos_tags:
                    if w.lower() == word:
                        pos_tag = pos
                        break
                if pos_tag:
                    break
        # If content is not a string (e.g., an integer), skip it
        else:
            continue
    word_info.append((word, pos_tag, freq))

# Create DataFrame
word_info_df = pd.DataFrame(word_info, columns=['Word', 'POS Tag', 'Frequency'])

# Display the DataFrame
word_info_df.head()

,Word,POS Tag,Frequency
0,3,None,460
1,1,None,1770
2,2,None,796
3,12,None,51
4,8,None,108
